In [31]:
import sqlite3
from dotenv import load_dotenv
import os
from google import genai
from google.genai import types

def get_connection():
    return sqlite3.connect('company.db')


load_dotenv()
api_key = os.getenv("LLM_HW_API_KEY")
if not api_key:
    raise ValueError("API Key not set")

client = genai.Client(api_key=api_key)


In [32]:
def add_employee(name, role, department, salary):
    conn = get_connection()
    cur = conn.cursor()

    cur.execute("""
        INSERT INTO employees (name, role, department, salary)
        VALUES (?, ?, ?, ?)
    """, (name, role, department, salary))

    conn.commit()
    conn.close()

In [33]:
def delete_employee(id):
    conn = get_connection()
    cur = conn.cursor()

    cur.execute("DELETE FROM employees WHERE id = ?", (id,))

    conn.commit()
    conn.close()

In [34]:
add_employee_tool_schema = {
    "name": "add_employee",
    "description": "Adds new employee in employee table with given data (name, role, department, salary)",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "Name of the employee.",
            },
            "role": {
                "type": "string",
                "description": "Role of employee in company",
            },
            "department": {
                "type": "string",
                "description": "Department where the employee works.",
            },
            "salary": {
                "type": "number",
                "description": "Employee's monthly salary.",
            },
        },
        "required": ["name", "role", "department", "salary"],
    },
}

In [35]:
delete_employee_tool_schema = {
    "name": "delete_employee",
    "description": "Deletes employee from employees table based on given id",
    "parameters": {
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "ID of the employee.",
            },
        },
        "required": ["id"],
    },
}

In [36]:
llm_tools = [
    add_employee_tool_schema,
    delete_employee_tool_schema
]

In [37]:
# Configure the client and tools
client = genai.Client(api_key=api_key)
tools = types.Tool(function_declarations=llm_tools) # type: ignore
config = types.GenerateContentConfig(tools=[tools])
chat = client.chats.create(model="gemini-2.5-flash", config=config)
def execute_agent(raw_query):
    response = chat.send_message(raw_query)
    if response.candidates[0].content.parts[0].function_call: # type: ignore
        function_call = response.candidates[0].content.parts[0].function_call # type: ignore
        print(f"Function to call: {function_call.name}")
        print(f"Arguments: {function_call.args}")

    
    

In [38]:
raw_query = input("Hello, What Can I do for you?: ")
execute_agent(raw_query)

Function to call: add_employee
Arguments: {'role': 'developer', 'salary': 5000, 'name': 'nika', 'department': 'sales'}
